# Summary

Folklore says that if we use more data, we'll make better estimates. In
today's lab, we'll qualify and quantify this statement. We're going to
look at how a few measures of the accuracy of our estimated curves vary
as a function of sample size. And to summarize what we see, we'll
estimate *rates of convergence*.

We'll use a few libraries.


In [ ]:
suppressPackageStartupMessages({
  library(tidyverse)
  library(CVXR)
})
CVXR::add_to_solver_blacklist('OSQP') 

theme_update(plot.background = element_rect(fill = "transparent", colour = NA),
		    panel.background = element_rect(fill = "transparent", colour = NA),
                    legend.background = element_rect(fill="transparent", colour = NA),
                    legend.box.background = element_rect(fill="transparent", colour = NA),
                    legend.key = element_rect(fill="transparent", colour = NA),
			panel.grid.major=element_line(color=rgb(1,0,0,.1,  maxColorValue=1)),
	        panel.grid.minor=element_line(color=rgb(0,0,1,.1,  maxColorValue=1)),
		    axis.ticks.x = element_blank(),
		    axis.ticks.y = element_blank(),
		    axis.text.x  = element_text(colour = "#aaaaaa"),
		    axis.text.y  = element_text(colour = "#aaaaaa"),
		    axis.title.x  = element_text(colour = "#aaaaaa"),
		    axis.title.y  = element_text(colour = "#aaaaaa", angle=90))

## Data

We'll start by looking at data sampled around a few curves $\mu$ on
$[0,1]$.

1.  A step, $\mu(x) = 1(x \ge .5)$.
2.  A line, $\mu(x)=x$.
3.  A step into a line, $\mu(x) = x 1(x \ge .5)$.
4.  A sine, $\mu(x)=\sin(\pi x)$.

Here's some code for evaluating our curves. We've put them in a list, so
`mu$step(x)` is $\mu(x)$ for the step function and so on.

In [ ]:
mu = list(step     = function(x) { 1*(x >= .5) },
	        line     = function(x) { x },
	        stepline = function(x) { x*(x >= .5) }, 
	        sin      = function(x) { sin(pi*x) })


We'll sample $X_i$ from the uniform distribution on $[0,1]$ and then
sample $Y_i$ around $\mu(X_i)$, taking $Y_i=\mu(X_i) + \varepsilon_i$
where $\varepsilon_i$ is independent of $X_i$ with mean zero. 

  - We'll start out by working with Gaussian noise. 
  - But if we have time, we'll also take a look at what changes when we have ...
    - lighter-tailed uniformly-distributed noise 
    - heavier-tailed $t_3$-distributed noise 

    Both of these will be scaled to have the same standard deviation $\sigma=\sqrt{\operatorname*{Var}{\varepsilon_i}}$.

Here's some code for sampling noise from these distributions. Again,
we've put them in a list, but now that list is the output of a function
of the noise standard deviation $\sigma$, so `noise(.5)$gaussian(n)`
gives us $n$ draws from the gaussian distribution with mean zero and
standard deviation $.5$. We'll set our default choice for $\sigma$
to $.5$.


In [ ]:
#| label: noise-distributions
noise = function(sigma) { 
    list(gaussian = function(n) { sigma*rnorm(n) },
	       uniform  = function(n) { sigma*sqrt(3)*runif(n,-1,1) },
	       t        = function(n) { (sigma/sqrt(3))*rt(n, 3) }) }

sigma = .5

Let's take a look at our curves and the data we're sampling around them
with these three different noise distributions. There are, of course,
differences from one noise distribution to another, but qualitatively
things look pretty similar.

In [ ]:
n = 100
X = runif(n)
x = seq(0,1,by=.001)

for(mm in 1:length(mu)) {
  observations = purrr::map_dfr(noise(sigma), .id='noise', function(rnoise) { 
    data.frame(X=X, Y=mu[[mm]](X) + rnoise(n))
  })   
  curve = purrr::map_dfr(noise(sigma), .id='noise', function(rnoise) {
    data.frame(x=x, y=mu[[mm]](x))
  }) # we do this so faceting works: 
     # the values of mu(x) must be repeated for each noise distribution.
  plt = ggplot() + 
          geom_point(aes(x=X, y=Y), alpha=.2, data=observations) + 
          geom_line(aes(x=x, y=y), data=curve) + 
          facet_grid(cols=vars(noise))
  print(plt  + xlab('') + ylab(''))
}

While it's hard to have intuition for how accurately we'll be able to
estimate a curve in a quantitative sense, we get somewhere by asking a
more qualitative question. We can ask whether, based on the observations
$X_i,Y_i=\mu(X_i)+\varepsilon_i$, we can identify which of our four
curves we're looking at. To get a sense of how accuracy in this sense
relates to sample size, give this exercise a shot.

::: {.callout-exercise}
Try varying sample size $n$ in the code above. Based on what you see, at
roughly what sample size do you start getting a clear sense of the shape
of the curve $\mu(x)$ from looking at the data? You don't have to be all
that precise: try doubling sample sizes $n=50,100,200,400,\ldots$. 

Your answer may vary by the curve $\mu$ and the noise distribution. Briefly
describe the trends. No need to make a big table for all curves and
noise distributions.
:::

::: {.callout-exercise}
Now let's test your intuition. We'll display, in random order, plots
like the ones above for our four different curves $\mu(x)$. See if you
can work out which is which.

Run the block below to plot our four curves in random order with meaningless
numbers 1-4 as labels. Write down which of our curves you think you're
seeing for each number. Then, run the block below that, which will show
the same plot with informative labels, to check your answers.

Change the first block to try different values of $n$ and report whether
your intuition from the last exercise was right. And repeat this for the
three noise distributions. To do this, change 'gaussian' in
`rnoise=noise(sigma)$gaussian` to 'uniform' or 't'.
:::

In [ ]:
n = 100
rnoise = noise(sigma)$gaussian
X = runif(n)
 
mu.order = sample(1:4)
observations = mu.order |> map(function(mm) { 
  data.frame(X=X, Y=mu[[mm]](X)+rnoise(n), curve=mm)
}) |> list_rbind()
plt = ggplot() + geom_point(aes(x=X, y=Y,), alpha=.2, data=observations)
plt + facet_wrap(~curve, nrow=2) + xlab('') + ylab('')

In [ ]:
informative_labeller = function(factors) { list(names(mu))[factors$curve] }
plt + facet_wrap(~curve, nrow=2, labeller=informative_labeller) + xlab('') + ylab('')

Now that we've gotten a sense of what sample sizes are required to
visually distinguish our four curves, let's start fitting them using monotone regression. 
We'll see how our choice of model impacts our ability to estimate them accurately.

In addition to linear regression, which chooses the best fitting curve
from the set of all lines, and monotone regression, which chooses from
the set of all increasing curves, we'll use a model-selection approach
that plays the same guessing game you've been playing. Just like you,
it'll choose from the set of four curves we're sampling our data around.
This is, of course, unusable in practice; it's never going to be
plausible that you know that the data's conditional mean
$\mu(x)=E[Y_i \mid X_i=x]$ is one of a few curves you know a-priori. But
comparing this to the other methods will give you a sense of how much
harder the task of estimating a curve via something like monotone
regression is than the task you were working on visually.

## Regression Code

We'll be comparing linear regression to monotone regression, so you'll
want monotone regression code. To minimize the time we wait for code to
run, we'll use R's built-in implementation, `isoreg`, which uses a fast
specialized algorithm called PAVA to solve the monotonicity-constrained
least squares problem. I'll give you a code that wraps it so it behaves
exactly like the function `monotonereg` we wrote in lab. I'll
also give you an implementation of `selectionreg`, which selects one of
our four curves as discussed above.

In [ ]:
monotonereg = function(X,Y, decreasing=FALSE) {
  isoreg.model = if(decreasing) { isoreg(-X,Y) } else { isoreg(X,Y) }
  mu.hat = array(dim=length(X))
  mu.hat[isoreg.model$ord] = isoreg.model$yf
  
  model = list(X=X, mu.hat=mu.hat)
  attr(model, "class") = "monotonereg"
  model
}

# make predictions based on piecewise-constant interpolation
# we use the curve that jumps at each observation and is otherwise constant
# that is, if X[1] < X[2] < ..., 
#   mu.hat(x) for x between X[k] and X[k+1] is mu.hat(X[k])   [case 1]
#             for x > X[k]  is mu.hat(X[k])                   [case 2]
#             for x < X[1]  is mu.hat(X[1])                   [case 3]
predict.piecewise.constant = function(model, newdata=data.frame(X=model$X)) {
  increasing.order = order(model$X) 
  X = model$X[increasing.order]
  mu.hat = model$mu.hat[increasing.order]
  
  # for each new data point x[k]
  # find the closest observed X[i[k]] left of x[k]
  # i.e., i[k] is the largest integer i for which X[i] <= x[k] 
  # this covers cases 1 and 2
  i = findInterval(newdata$X, X) 
  # if there is no X[i] < x[k], findInterval sets i[k]=0
  # to cover case 3, we want X[i] for i=1 when this happens.
  i[i==0] = 1
  # report the values of mu.hat(X[k]), one for each x
  mu.hat[i]
}
predict.monotonereg = predict.piecewise.constant

In [ ]:
selectionreg = function(X,Y) {
  mses = sapply(mu, function(m) { mean((Y-m(X))^2) })
  model = list(X=X, mu=names(mu)[which.min(mses)])
  attr(model, 'class') = 'selectionreg'
  model
}
predict.selectionreg = function(model, newdata=data.frame(X=model$X)) {
  mu[[model$mu]](newdata$X)
}

Often, having our code's notation match our mathematical notation makes
it easier to read. To help with that, here's a function that takes a
model like we'd get by calling `lm`, `monotonereg`, or `selectionreg`
and gives us back a function $\hat\mu$ that makes predictions
$\hat\mu(x)$ using the model.

In [ ]:
prediction.function = function(model) { 
  function(x) { predict(model, newdata=data.frame(X=x)) } 
}

Here's how we use it.

In [ ]:
X = runif(n)
Y = mu$step(X) + noise(sigma)$gaussian(n)

mu.hat = prediction.function(monotonereg(X,Y))
mu.hat(c(0,.5,1))

And plot using it.

In [ ]:
x=seq(0,1,by=.001)
ggplot() + geom_point(aes(x=X, y=Y), alpha=.2) + 
           geom_line(aes(x=x, y=mu.hat(x)))

# Squared Errors

To get a sense of how sample size impacts the accuracy of our estimator,
we'll vary sample size and track a few measures of its error. We'll look
at three error measures.

1.  Sample mean squared error, the estimator's typical level of error on
    the sample used to fit the curve.
    $$ \lVert\hat\mu-\mu\rVert_{L_2(P_n)}^2
                     = \frac{1}{n}\sum_{i=1}^n \{ \hat\mu(X_i) - \mu(X_i)\}^2. $$

2.  Population mean squared error, its typical level error on points
    drawn independently from the same distribution as that sample. For
    $\tilde X$ independent of $X_1 \ldots X_n$ with the same distribution,
    $$ \lVert\hat\mu-\mu\rVert_{L_2(P)}^2 
                     = E\left[ \{ \hat \mu(\tilde X) - \mu(\tilde X) \}^2 \mid X_1 \ldots X_n\right]. $$
    
    - Since we're sampling $X_i$ from the uniform distribution, that's
      $\int_0^1\{ \hat \mu(x) - \mu(x) \}^2 dx$. We won't calculate this integral exactly.
    - To approximate it, we'll use the simplest numerical integration scheme there is. We'll
    just use the average of $\{\hat\mu(x)-\mu(x)\}^2$ for an evenly-spaced grid of points $x \in [0,1]$.

3.  Squared error at a single point $x$,
    $$
    \{ \hat \mu(x) - \mu(x) \}^2
    $$. 
    We'll look at the point $x=0$ at the left edge of $X_i$'s distribution. 
     
     - This is the sort of thing you wind up working with in RDD estimates.[^1]

[^1]: Think of it like this. You've taken the data to the right of the
    enrollment cutoff in our class size example (enrollment $> 40$),
    defined $X_i$ by shifting and scaling enrollment so the cutoff
    occurs at $x=0$ so the cutoff occurs at $x=0$ and all $X_i$ lie in
    into the range $[0,1]$ ($X_i=(\text{enrollment}_i-40)/40$), fit the
    data, and are now evaluating it at $X_i=0$ to get the estimate
    $\hat \mu_{\text{right}}(40)$. We'll often want to scale the data
    into the unit interval like this for other reasons, so this is a
    more natural way of thinking about things than it may sound now.

We'll do this for three estimators.

1. the least squares line (we use the `R` built-in `lm`) 
2. the monotone regression estimator (we use the function `monotonereg` defined above)
3. the estimator that selects one of our four choices of $\mu$ (we use `selectionreg` defined
above).

This kind of code is kind of a pain to write, so I'm going to give you
everything you need. But you'll be using this code in a few homeworks including this week's, so you may
want to take a look at the last section of this notebook, where I describe how the code works.


::: {.callout-exercise}
Run the code below to plot these three error measures at a range of
sample sizes. As it is, it plots them at sample sizes
$n \in \{25,50,100,200\}$. Feel free to add some more. Briefly describe
what you see. 

Later on, I'll ask more precise questions, so don't worry
about being complete.
:::


Here, to start, is some code for computing our three error measures
given a function $\hat\mu(x)$. Note that so we can substitute one for
another in our code, they all take the sample $X=X_1 \ldots X_n$ as an
argument even though only sample MSE actually needs it.

In [ ]:
sample.mse     = function(mu.hat, mu, X) { 
  mean((mu.hat(X) - mu(X))^2) 
}
population.mse = function(mu.hat, mu, X) { 
  x=seq(0,1,by=.001)
  mean((mu.hat(x) - mu(x))^2) 
}   
point.se = function(mu.hat, mu, X) { 
  (mu.hat(0) - mu(0))^2 
}

And here's a code for making a table of these 3 errors, for our 3
regression methods, at our 4 different sample sizes. Naturally, it has
$3 \times 3 \times 4 = 36$ rows, one for each combination of these
things

In [ ]:
errors = list(sample=sample.mse, population=population.mse, point=point.se) 
models = list(lines = function(X,Y) { lm(Y~X, data.frame(X=X,Y=Y)) },
              monotone = monotonereg,
              selection  = selectionreg)

tabulate.errors = function(X,Y, mu, ns) { 
  map_dfr(ns, function(n) {
    map_dfr(models, .id='model', function(model.fit) {
	    model = model.fit(X[1:n], Y[1:n])
	    map_dfr(errors, .id='error.measure', function(error) {
        data.frame(n=n, error=error(prediction.function(model), mu, X[1:n]))
	    })
    })
  })
}


We'll run it to produce tables for all combinations of the four curves
$\mu(x)$ and three noise distributions described above, then stack them all together. If you want, you can look at the
table, but it's probably easier to plot what's in it and look at the
plots.


In [ ]:
ns = c(25,50,100,200)
X = runif(max(ns))

tab = map_dfr(mu, .id='mu', function(mu) {
        map_dfr(noise(sigma), .id='noise', function(rnoise) {
          Y = mu(X) + rnoise(length(X))
          tabulate.errors(X, Y, mu, ns)
        })
    })

tab


Here's a plot of population and sample MSE. We'll just look at what
happens with gaussian noise.


In [ ]:
noise.type = 'gaussian'
noise.tab = tab[tab$noise==noise.type, ]

ggplot(noise.tab[noise.tab$error.measure != 'point',], 
       aes(x=n, y=error, color=model, linetype=error.measure, shape=error.measure)) + 
  geom_line() + geom_point() + facet_grid(cols=vars(mu), )  


And here's a plot of squared error at the point $x=0$.


In [ ]:
ggplot(noise.tab[noise.tab$error.measure == 'point', ], 
       aes(x=n, y=error, color=model, linetype=error.measure, shape=error.measure)) + 
    geom_line() + geom_point() + facet_grid(cols=vars(mu), rows=vars(noise))

::: {.callout-exercise}
Ignore the squared error at the point $x=0$ for now. Answer these
questions for the linear model, the the monotone (increasing) model, and
the select one-of-four model. 

1. Is the qualitative behavior you see roughly the same for all curves? 
2. If not, for which curve or curves is it different? 
3. How are they different? Why?

**Tip.** If you see different behaviors in the error plots but don't
know why, plot the fitted curve $\hat \mu$ on top of the data and look
at it.

:::


::: {.callout-exercise}
Again, ignore squared error at the point $x=0$. Is either sample or
population mean squared error typically larger or is their relationship
inconsistent? Why do you think that's the case?
:::

::: {.callout-exercise}
Now let's think about why we've been ignoring squared error at $x=0$.
The errors we have plotted are *random*. They depend on the sample
$\mathcal{S}=\{(X_i,Y_i) : i \le n\}$ used to fit $\hat\mu$.[^2] Rerun
the code in this section to draw a new sample and make your plots. Do it
a few times. Is the qualitative behavior you've already described
consistent from sample to sample? What about squared error at $x=0$? In
which cases is it inconsistent? Why?


**Tip.** Each time you resample, plot the fitted curves $\hat \mu$ on
top of the data and look at them.
:::

[^2]: And, in the case of sample mean squared error, also more directly.



# Expected Squared Error

In this section, we're going to summarize the distribution of the mean
squared errors we plotted in the previous one. Because each error is a
function of the sample, $\text{error}(\mathcal{S})$, it makes sense to
talk about its mean $\text{E}[ \text{error}(\mathcal{S})]$ and its
variance $\text{Var}[ \text{error}(\mathcal{S})]$. We can, of course,
estimate those by the sample mean and sample variance, 

$$
\hat{\text{E}}[ \text{error}(\mathcal{S})] := \frac{1}{R}\sum_{r=1}^R \text{error}(\mathcal{S}_r)
\quad \text{ and } \quad
\widehat{\text{Var}}[ \text{error}(\mathcal{S})] := \frac{1}{R}\sum_{r=1}^R \left\{\text{error}(\mathcal{S}_r) - \hat{\text{E}}[ \text{error}(\mathcal{S})] \right\}^2
$$ 

based on independent replications
$\mathcal{S}_1 \ldots \mathcal{S}_R$ of the sample. And, as a
back-of-the-envelope thing, you might think of
$\hat{\text{E}}[\text{error}(\mathcal{S})] \pm 2\sqrt{\widehat{\text{Var}}[\text{error}(\mathcal{S})]}$
as a 95% confidence interval for $\text{E}[\text{error}(\mathcal{S})]$, i.e. an interval you might
expect 95% of replications of $\text{E}[\text{error}(\mathcal{S})]$ to be in.

To show this distributional summary, we'll replicate the plots from
section on squared errors above, with each point being the mean
$\hat{\text{E}}[\text{error}(\mathcal{S})]$ over $R=100$ replications.
Then, to give a sense of spread, we'll display our prediction interval
by plotting error bars extending two estimated standard deviations
$\sqrt{\widehat{\text{Var}}[\text{error}(\mathcal{S})]}$ in each
direction.

First, we'll make a $R$ tables like we did before, one for each
replication $\mathcal{S}_r$ of our the dataset, and stack them. This
takes is a bit slow, but it takes under a minute on my laptop. If it's
too slow on yours, you can decrease the number of replications $R$ to
reduce the runtime. This code should run for about $R$ times the amount
of time it took to make the table above.

In [ ]:
R=100
rep.tab = map_dfr(1:R, function(rep) { 
            map_dfr(mu, .id='mu', function(mu) {
              map_dfr(noise(sigma), .id='noise', function(rnoise) {
                Y = mu(X) + rnoise(length(X))
                tabulate.errors(X, Y, mu, ns)
              })
            })
          })



Then we'll do our plots. `ggplot`'s function `stat_summary` does a lot
of the work for us, aggregating the data into the means and the upper
and lower confidence bounds we need to plot error bars. Here's the code.
We'll plot sample and population MSE together as we did before so
they're easy to compare, but because the error bars make that plot a
little crowded, we'll also plot them separately.


In [ ]:
mse.with.errorbars.plot = function(tab) {
  ggplot(tab, aes(x=n, y=error, color=model, linetype=error.measure, shape=error.measure)) + 
    stat_summary(geom='line', fun=mean) +
    stat_summary(geom='pointrange', fun=mean,
     fun.min=function(x){ mean(x)-2*sd(x) }, 
		 fun.max=function(x){ mean(x)+2*sd(x) },
		 position=position_dodge(20)) +
     facet_grid(cols=vars(mu), rows=vars(noise)) 
}

rep.noise.tab = rep.tab[rep.tab$noise==noise.type,]
mse.with.errorbars.plot(rep.noise.tab[rep.noise.tab$error.measure!='point',])
mse.with.errorbars.plot(rep.noise.tab[rep.noise.tab$error.measure=='sample',])
mse.with.errorbars.plot(rep.noise.tab[rep.noise.tab$error.measure=='population',])
mse.with.errorbars.plot(rep.noise.tab[rep.noise.tab$error.measure=='point',])

::: {.callout-exercise}
Looking at these plots, revisit your answers from the previous part.
Describe what you're seeing. 

- Maybe you got some answers that were true for the particular sample $\mathcal{S}$ you were looking at, but aren't true on average over samples.
- Maybe you got some answers that were true on average, but see a lot of variation from sample to sample.
:::


# Rates of Convergence

The plots of error vs. sample size in the sections above contain a lot of information. 
Some things are easy enough to see. It's pretty clear when errors aren't decreasing
to zero because we're using a model that can't fit the curve $\mu$.
However, it can be hard to see more quantitative phenomena, like how
much faster one model's errors are converging to zero than another's, as
this involves behavior at a wide range of $n$. It's a scale problem. If
you can see the curve at all for $n=50$, you're zoomed out far enough
that you can't see what's going on for $n \ge 200$. To summarize the
curves in a way that can help make some of these relationships clear, we
often use *rates of convergence*. We'll say something like $\hat \mu$
converges to $\mu$ at the rate $n^{-\beta}$, by which we mean that if we calculate its expected 
population mean squared error $\text{MSE} = E \lVert \hat \mu - \mu \rVert_{L_2(P)}^2$ at increasing 
sample sizes, then $\text{RMSE}=\sqrt{MSE}$ satisfies a bound $\text{RMSE} \le \alpha n^{-\beta}$ for some 
constant $\alpha$.[^3] [^4] We can make analogous statements for sample mean squared
error and squared error at the point $x=0$. 

[^3]: This is a bit imprecise. The precise language we use is a
    mouthful: $\hat \mu$ converges *in mean-square* to $\mu$ in the
    $L_2(P)$ norm.
[^4]: We tend to talk about $\text{RMSE}$ rather than $\text{MSE}$ because it's a bit more intuitive. You can think of it as a typical distance between $\hat \mu - \mu$, which is something you could see in a plot of the curves $\hat\mu$ and $\mu$, rather than the square of that, which
you'd have to calculate.

Naturally, in any fixed sample size the rate $n^{-\beta}$ is meaningless
by itself because we could just increase the constant $\alpha$ to get a
bound no matter what $\text{RMSE}$ is. What makes this concept
meaningful is the requirement that the constant be the same as sample
size gets bigger. 


Here's a way to interpret this, although it does require that our rate
be an approximation to rather than an upper bound on our
root-mean-squared error (RMSE), i.e., that
$\text{RMSE} \approx \alpha n^{-\beta}$. If our rate is
$n^{-1/2}$, then if our error at sample size $100$ is roughly $.1$, your
error at sample size $10,000=100^2$ would be roughly $.01$, as the ratio
of errors should be roughly this. 

$$ 
\frac{\text{RMSE}_{100^2}}{\text{RMSE}_{100^1}} 
  \approx \frac{\alpha (100^2)^{-1/2}}{\alpha (100^1)^{-1/2}} 
  = \left( \frac{100^2}{100^1} \right)^{-1/2} 
  = 100^{-1/2} = .1 
$$

More generally, if our rate is $n^{-\beta}$, then if we increase our
sample size by a factor of $10^{1/\beta}$, we get an extra digit of
precision, i.e., for our error to go from $.1$ to $.01$ or from $.03$ to
$.003$. To get an extra digit ...

  - we need $100$ times more data if our rate is $n^{-1/2}$.
  - we need $1000$ times more data if our rate is $n^{-1/3}$.
  - we need $10,000$ times more data if our rate is $n^{-1/4}$.

We tend not to talk less about rates worse than $n^{-1/4}$ because it's really hard to get
$10,000$ times more data. And we tend to talk about rates more in the
context of theory than in empirical studies like this one. But getting
some experience with them now should help build intuition that will help
us make sense of theoretically-derived rates of convergence, which we'll
talk a lot about later in the semester.

To get a sense of each estimator's rates of convergence to the curves
$\mu$, we'll fit a simple two-parameter exponential model
$m(n)=\alpha n^{-\beta}$ to predict the square root of the
expected squared error estimates we calculated in
the last section. To do this, we'll use the `R` built-in for nonlinear regression, `nls`. 
And to make sure this exponential model is meaningful, we'll check its the
fit visually by adding the predicted MSE $\hat m(n)^2=\hat\alpha^2 n^{-2\hat\beta}$
it to the plots of the actual expected error curves.

**Sample size**.
The range of samples sizes we've been working with so far,
$n=\{25, 50,100,200\}$, is a little small to estimate a rate accurately.
So we'll make a table using a larger range here, doubling from 25 to
1600. Other than changing that, this is exactly the code we ran a moment ago.

In [ ]:
R=100
ns.rate = 25*2^(0:6)
X = runif(max(ns.rate))

rnoise = noise(sigma)[[noise.type]]
rate.tab = map_dfr(1:R, function(rep) { 
            map_dfr(mu, .id='mu', function(mu) {
              Y = mu(X) + rnoise(length(X))
              tabulate.errors(X, Y, mu, ns.rate)
            })
        })

In [ ]:
# adding 1e-2/log(log(n)) to our errors below is a hack that we need to 
# make nls tolerate working on the errors of our selection estimator, 
# which are often zero. This replaces those zeros with a very slowly 
# decreasing curve, so we'll slightly overestimate our rate of convergence
hacked.rate.tab = rate.tab
hacked.rate.tab[rate.tab$model=='selection','error'] = 
  rate.tab[rate.tab$model=='selection','error'] + 
  1e-2/log(log(rate.tab[rate.tab$model=='selection','n']))

rates = hacked.rate.tab |> group_by(error.measure, model, mu) %>% 
		group_modify(function(group,...) {
		    reg.data = group |> group_by(n) |> summarize(error=mean(error))
		    model = nls(formula = sqrt(error) ~ a*n^(-b), data=reg.data, 
		                start=list(a=1, b=1/2))
		    data.frame(a=coef(model)[1], b=coef(model)[2])
	    })
rates[rates$error.measure=='population', ]
rates[rates$error.measure=='sample', ]
rates[rates$error.measure=='point', ]

In [ ]:
rate.predicted = hacked.rate.tab |> group_by(mu,model,error.measure) |>
		group_modify(function(group,...) {
		    reg.data = group |> group_by(n) |> summarize(error=mean(error))
		    model = nls(formula = sqrt(error) ~ a*n^(-b), data=reg.data, 
		                start=list(a=1, b=1/2))
		    reg.data$error = predict(model)^2
		    reg.data
	    })

rate.plots = map(unique(rate.tab$error.measure), function(measure) {
  mse.with.errorbars.plot(rate.tab[rate.tab$error.measure == measure,]) + 
          geom_line(aes(x=n, y=error, color=model), alpha=.4, linewidth=2, 
				            data=rate.predicted[rate.predicted$error.measure == measure, ]) +
			    facet_grid(cols=vars(mu), rows=vars(error.measure))
})

**Styling**.
Here's the comparison of the error prediction curves we get using
our polynomial model $\text{RMSE} \approx \alpha n^{-\beta}$
and the actual error curves.
I've plotted the predictions curves as lines that matching the actual 
error curve in color, but wider and semi-transparent. That means that
if the predictions are really good, what we'll see are thin lines with 
wide 'tubes' around them. You'll see. 

In [ ]:
walk(rate.plots, print) 

Now let's think about what this all means.

::: {.callout-exercise}
Let's start with the line, since all of our models can fit it. Have you
estimated different rates of convergence $n^{-\beta}$ in population mean
squared error for the lines and monotone regression models?[^4] What do you make
of this? Would you say that this summary was helpful or misleading? Why?
What about sample mean squared error and squared error at the point
$x=0$?

**Tip.** It may be helpful to think about the values of $\alpha$ you've
estimated.
:::

[^4]: I left out the select-one-of-four model because the idea of a rate
    for it is a little weird. Error is, after all, zero for most or all
    replications for most $n$ we consider. To get at what's happening
    with this estimator, we need to be talking about something like the
    smallest sample size at which we tend to get zero error.


::: {.callout-exercise}
Now let's think about the other three curves $\mu$ that you've fit: the
step, the stepline, and the sine. Are you seeing the same rates of
convergence from linear and monotone regression that you got in the
previous exercise, when $\mu$ was the line? Does your answer depend on
$\mu$? If so, try to explain why you're seeing this variation.
:::

# Universality

This is optional. Do it if you've got extra time. I'm not going to
provide solutions for these, but we'll dig into these issues a little
later on when we've got some theory to guide us.

When we looked at the data, we saw that it looked roughly the same no
matter whether we used gaussian, uniform, or t-distributed noise. So
you'd expect that the MSE and rates of convergence you see wouldn't
change much.

::: {.callout-exercise}
Change the distribution of the noise in the code above and rerun it.
That's as easy as changing `noise.type='gaussian'` in the plot-pop-mse
block to `noise.type='t'` or `noise.type='uniform'` and rerunning
everything below that. Discuss whether you see any changes.
:::

The data certainly would look different if we increased the scale of the
noise, say from having standard deviation $0.5$ as we've been using to
having standard deviation $5$. However, it might be the case that having
a little data with a little noise is similar to having a lot of data
with a lot of noise. Look into whether that's the case.

::: {.callout-exercise}
Increase the standard deviation of the noise. All you have to do is
change `sigma = .5` in the noise-distributions block above to `sigma=5`. 
While the quantitative result you see
will change, i.e. you'll have more error, you might expect the
qualitative, e.g. the relationships between types of MSE and the rates
of convergence, to stay the same. Discuss what you find. Then repeat,
instead decreasing the standard deviation of the noise, e.g. using
`sigma=.05`.
:::


# Using Tidyverse Tools

While I don't often shill for the tidyverse, I found those tools pretty
helpful here. If you're not familiar, the 'tidy' way to do things is to
collect a bunch of stuff in one big dataframe. When I tabulated errors
earlier, I put them in a data frame with columns `error`, `error.measure`, `n`, `mu`, `model`, and `noise`. 
This makes everything very easy to plot using ggplot. I'm using these
commands to plot mine. I'm plotting the mean squared errors and squared
error at the point $x=0$ separately because the scales tend to be
different.

In [ ]:
ggplot(tab[tab$error.measure != 'point',], 
       aes(x=n, y=error, color=model, linetype=error.measure, shape=error.measure)) + 
  geom_line() + geom_point() + facet_grid(cols=vars(mu), rows=vars(noise))  

In [ ]:
ggplot(tab[tab$error.measure == 'point', ], 
       aes(x=n, y=error, color=model, linetype=error.measure, shape=error.measure)) + 
    geom_line() + geom_point() + facet_grid(cols=vars(mu), rows=vars(noise))


My code leans heavily on the function `map_dfr` in the tidyverse package **purrr**.^[It turns out that this function is deprecated, i.e. the maintainers of the package **purrr** are recommending that you stop using it because they're planning to remove it from the package at some point in the future. It's usually considered bad form to use deprecated functions, but it's really hard to avoid when you use the tidyverse, so I'm giving you code that does it. The package maintainers in the tidyverse group are always renaming functions and moving them from package to package with tiny changes, so if you wrote code using tidyverse stuff a few years ago, the odds are pretty good something you've used is deprecated at this point. That's usually considered bad form too, and because of that, many people I know have given up on the idea of using tidyverse stuff in any code that isn't essentially disposable. But there's a lot of useful stuff in there, so I try to take a more conservative approach unless I really need my code not to break, e.g. when I'm writing packages of my own that lots of people will be using. I don't really worry about tidyverse functions getting deprecated because it takes a while for them to actually go away and when they do, you can usually redefine the old functions in terms of the new ones with at most a couple lines of code.] 

This is really nothing special. It's just like Python's map or R's built-in lapply, except that
it expects the function you're mapping to return a dataframe, and
instead of collecting these dataframes in a list, it stacks them
vertically into a single dataframe. It takes two arguments.

1.  a list `list(a=x,b=y,c=z,...)`

2.  a function `f(x)` that outputs a data.frame

and returns `f(x)`, `f(y)`, `f(z)`, ... vertically stacked into a data
frame `df`. And it has one other slightly convenient feature. If you
pass the option .id='somestring' for some string id, it'll add a column
`df$somestring` that stores the names `a`,`b`,`c`, ... in the rows
corresponding to `f(x)`, `f(y)`, and `f(z)` respectively.

You can use it to loop over models, error measures, curves, etc. You can
nest calls to loop over multiple things; everything winds up getting
stacked together.

Here's some example code that might help a bit.


In [ ]:
mus=list(a=1, b=2, c=3)
sigmas=list(small=.1, big=10)
mean.squares = map_dfr(mus, .id='mu', function(mu) {
                  map_dfr(sigmas, .id='sigma', function(sigma) {
                    Y = mu + sigma*rnorm(10)
                    data.frame(mean.square=mean(Y^2))
                  })
               })
mean.squares